In [ ]:
#몇 가지의 측정 데이터가 주어졌을 때, 주택 가격에 대한 예측
#housing에서 데이터 탐색과 시각화를 했고, 여기서는 그 후로 예측하는것.
#다변량 회귀문제, 데이터의 연속적인 흐름이 없고, 데이터가 메모리에 들어갈 만큼 충분히 작음 => 미니배치 사용X
#성능 측정 지표는 RMSE(평균제곱근 오차) 사용

In [2]:
#데이터 추출. 그냥 다운받는것보다 코드로 하면 자동화되서 편하다고 함.
#디렉터리에 압축을 풀어서 housing.csv파일 생성

import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing/tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH) :
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    

In [5]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()

In [6]:
#테스트 세트 만들기_1

import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

train_set, test_set = split_train_test(housing, 0.2) 

#20퍼센트의 테스트 세트 생성
# print(len(train_set), "train +", len(test_set), "test")

16512 train + 4128 test


In [8]:
#테스트 세트 만들기_2
#위와 같이 테스트 세트를 만들어서 반복하면, 테스트 세트의 의미가 없어짐(전체 데이터 셋을 보는거랑 마찬가지이므로)

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [11]:
#total_bedrooms 특성에는 값이 조금 누락되어있음. 이것을 전처리해줘야함.
#사이킷런의 imputer는 누락된 값을 특성의 중간값으로 대체함

from sklearn.preprocessing import Imputer

imputer = Imputer(strategy="median")

#"ocean_proximity는 텍스트 특성이기 떄문에 잠시 빼둠"
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

X = imputer.transform(housing_num)

#위의 결과값은 넘파이 배열인데, 이것을 다시 판다스 프레임으로 되돌림.
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index = list(housing.index.values))

In [16]:
#위에서 텍스트 특성이기 때문에 잠시 빼둔 ocean_proximity를 처리해주어야 함.
#ocean_proximity는 텍스트, 범주형 특성인데 이를 one_hot encoding을 통해 숫자로 바꿈

housing_cat = housing["ocean_proximity"]

housing_cat_encoded, housing_categories = housing_cat.factorize() #factorize() : 각 카테고리를 다른 정수값으로 매핑해줌
#이 코드의 문제는, 값이 1과 2면 컴퓨터가 연관이 있다고 생각하는데, 사실 1과 4가 더 연관있고...
#숫자의 의미를 없애기 위해(??) one_hot_encoding사용

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1, 1))
housing_cat_1hot = housing_cat_1hot.toarray()
